In [44]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score


In [2]:
food = pd.read_csv("en.openfoodfacts.org.products.tsv", sep='\t',low_memory=False)

In [479]:
food2 = food[np.isfinite(food['energy_100g'])]
food2 = food2[np.isfinite(food2['fat_100g'])]
food2 = food2[np.isfinite(food2['saturated-fat_100g'])]
food2 = food2[np.isfinite(food2['monounsaturated-fat_100g'])]
food2 = food2[np.isfinite(food2['polyunsaturated-fat_100g'])]
food2 = food2[np.isfinite(food2['carbohydrates_100g'])]
food2 = food2[np.isfinite(food2['sugars_100g'])]
food2 = food2[np.isfinite(food2['fiber_100g'])]
food2 = food2[np.isfinite(food2['proteins_100g'])]
food2 = food2[np.isfinite(food2['sodium_100g'])]

dff = food2[['product_name', 'generic_name', 'countries_en', 'energy_100g', 
       'fat_100g', 'saturated-fat_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g',
       'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g', 'salt_100g',
       'ingredients_from_palm_oil_n', 'cholesterol_100g', 'trans-fat_100g', 'iron_100g', 'calcium_100g',
       'vitamin-a_100g', 'vitamin-c_100g',]]

dff['product_name'] = dff['product_name'].str.lower()
dff = dff[pd.notnull(dff['product_name'])]
dff = dff[pd.notnull(dff['countries_en'])]
dff['healthy'] = 0

/Users/giordun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [478]:
dff.loc[dff.product_name.str.contains('tuna'), 'product_name'].head()

4669                 chunk light tuna in oil
12326      premium chunk light tuna in water
13134              chunk light yellofin tuna
13135    chunk light yellowfin tuna in water
24895      premium yellowfin steaks ahi tuna
Name: product_name, dtype: object

In [480]:
# Test, this person wants to build a muffin diet

# Muffin Choices
#choices = [2254, 2255, 2256, 281838, 150993, 103546, 2765, 112164, 2323, 149893, 17435, 98391, 2106]

# Tuna
# choices = [168835, 151484, 164299, 130003, 129181, 59334, 59328, 55469, 31442, 27837, 25484, 25482, 24895, 13135, 4669]

# Tuna Muffin
choices = [2254, 2255, 2256, 281838, 150993, 103546, 2765, 112164, 2323, 149893, 17435, 98391, 2106, 168835, 151484, 164299, 130003, 129181, 59334, 59328, 55469, 31442, 27837, 25484, 25482, 24895, 13135, 4669]

dff.loc[choices, 'healthy'] = 1

In [481]:
# Prepping data for split, fill NaNs with medians

X = dff.iloc[:, 2:-1]
X = X.iloc[:,1:].apply(lambda x: x.fillna(round(x.median(),0)),axis=0)
y = dff.healthy

In [482]:
def summary_stats(clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('Accuracy - %.3f' % accuracy_score(y_test, y_pred))
    print('Precision - %.3f' % metrics.precision_score(y_test, y_pred, average = 'macro'))
    print('Recall - %.3f' % metrics.recall_score(y_test, y_pred, average = 'macro'))
    print('F1 score - %.3f' % metrics.f1_score(y_test, y_pred, average = 'macro'))
    print('\n')

In [471]:
print('Random Forestb')
summary_stats(RandomForestClassifier(class_weight = 'balanced'))

print('Random Foresta')
summary_stats(RandomForestClassifier(class_weight = 'auto'))

print('Decision Tree')
summary_stats(RandomForestClassifier())

Random Forestb
Accuracy - 0.964
Precision - 0.510
Recall - 0.982
F1 score - 0.511


Random Foresta
Accuracy - 0.973
Precision - 0.514
Recall - 0.986
F1 score - 0.520


Decision Tree
Accuracy - 0.977
Precision - 0.516
Recall - 0.989
F1 score - 0.526




In [472]:
print('kNN')
summary_stats(KNeighborsClassifier(3))

print('Logistic Regression')
summary_stats(LogisticRegression())

print('Gaussian Naive Bayes')
summary_stats(GaussianNB())

print('SVC')
summary_stats(SVC())

print('Decision Tree')
summary_stats(DecisionTreeClassifier())

print('Random Forest')
summary_stats(RandomForestClassifier())

kNN
Accuracy - 0.944
Precision - 0.505
Recall - 0.872
F1 score - 0.496


Logistic Regression
Accuracy - 0.945
Precision - 0.507
Recall - 0.972
F1 score - 0.499


Gaussian Naive Bayes
Accuracy - 0.972
Precision - 0.512
Recall - 0.953
F1 score - 0.517


SVC
Accuracy - 0.997
Precision - 0.606
Recall - 0.965
F1 score - 0.672


Decision Tree
Accuracy - 0.892
Precision - 0.504
Recall - 0.946
F1 score - 0.478


Random Forest
Accuracy - 0.974
Precision - 0.514
Recall - 0.987
F1 score - 0.521




In [483]:
import random
nonchoice = [X.loc[set(X.index) - set(choices)].index]

comp = random.sample(X.loc[set(X.index) - set(choices)].index.tolist(), 3*len(choices))

X.loc[comp]

X_train = pd.concat([X.loc[choices], X.loc[comp]])
y_train = pd.concat([y.loc[choices], y.loc[comp]])

X_test = X.loc[set(dff.index) - set(comp)]
y_test = y.loc[set(dff.index) - set(comp)]

In [484]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

test_full = dff.loc[X_test.index]
test_full['pred'] = y_pred

In [485]:
test_full[(test_full['healthy'] == 0) & (test_full['pred'] == 1)]

,product_name,generic_name,countries_en,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,carbohydrates_100g,sugars_100g,...,salt_100g,ingredients_from_palm_oil_n,cholesterol_100g,trans-fat_100g,iron_100g,calcium_100g,vitamin-a_100g,vitamin-c_100g,healthy,pred
131945,"true goodness, organic cream cheese",NaN,United States,1343.0,32.14,21.43,10.71,0.00,7.14,7.14,...,0.86106,0.0,0.107,0.00,0.00000,0.071,0.000429,0.0000,0,1
132092,pre-cut chicken breast chunks with rib meat,NaN,United States,372.0,2.23,0.89,0.45,0.00,0.00,0.00,...,0.72644,0.0,0.049,0.00,0.00064,0.000,0.000000,0.0000,0,1
132409,combat crunch protein bar,NaN,United States,1393.0,11.11,6.35,3.17,1.59,44.44,7.94,...,1.85166,0.0,0.062,0.00,0.00029,0.349,0.000000,0.0010,0,1
133138,large grade aa 5 dozen eggs,NaN,United States,586.0,9.00,3.00,4.00,1.00,2.00,0.00,...,0.33000,0.0,0.430,0.00,0.00144,0.040,0.000180,0.0000,0,1
2085,deviled egg potato salad,NaN,United States,837.0,13.57,2.14,3.57,7.14,15.71,4.29,...,1.14300,0.0,0.071,0.00,0.00051,0.014,0.000043,0.1500,0,1
2099,mozzarella sliced cheese,NaN,United States,1142.0,18.18,9.09,4.55,0.00,0.00,0.00,...,1.38430,0.0,0.045,0.00,0.00000,0.682,0.000273,0.0000,0,1
2107,"bakery fresh goodness, blueberry muffins, topp...",NaN,United States,1778.0,19.81,3.77,4.72,10.38,56.60,33.02,...,0.88646,0.0,0.066,0.00,0.00255,0.019,0.000028,0.0011,0,1
2108,honey raisin bran muffins,NaN,United States,1854.0,23.58,3.77,6.60,13.21,53.77,29.25,...,0.71882,0.0,0.052,0.00,0.00170,0.038,0.000028,0.0000,0,1
2228,pumpkin loaf cake,NaN,United States,1582.0,15.56,3.33,4.44,6.67,51.11,24.44,...,0.96012,0.0,0.056,0.00,0.00160,0.000,0.000267,0.0000,0,1
2249,"pudding cake, triple berry",NaN,United States,1632.0,19.48,3.25,5.19,10.39,50.65,32.47,...,0.75946,0.0,0.065,0.00,0.00140,0.052,0.000039,0.0031,0,1


In [455]:
test_full[(test_full['healthy'] == 1) & (test_full['pred'] == 1)]

,product_name,generic_name,countries_en,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,carbohydrates_100g,sugars_100g,...,salt_100g,ingredients_from_palm_oil_n,cholesterol_100g,trans-fat_100g,iron_100g,calcium_100g,vitamin-a_100g,vitamin-c_100g,healthy,pred
2106,"bakery fresh goodness, banana muffins, banana",NaN,United States,1778.0,21.70,4.72,5.66,11.32,50.94,26.42,...,0.90932,0.0,0.071,0.0,0.00255,0.019,0.000057,0.0000,1,1
2254,blueberry muffins,NaN,United States,1778.0,19.81,3.77,4.72,10.38,56.60,33.02,...,0.88646,0.0,0.066,0.0,0.00255,0.019,0.000028,0.0011,1,1
2255,banana nut muffins,NaN,United States,1778.0,21.70,4.72,5.66,11.32,50.94,26.42,...,0.90932,0.0,0.071,0.0,0.00255,0.019,0.000057,0.0000,1,1
2256,double chocolate muffins topped with chocolate...,NaN,United States,1895.0,24.53,8.49,5.66,10.38,52.83,33.96,...,0.86360,0.0,0.080,0.0,0.00255,0.019,0.000085,0.0000,1,1
2323,"lemon poppy seed muffins, lemon poppy seed",NaN,United States,1778.0,20.75,4.25,5.66,10.38,54.72,30.19,...,0.88646,0.0,0.066,0.0,0.00255,0.019,0.000028,0.0011,1,1
2765,double chocolate muffins topped with chocolate...,NaN,United States,1895.0,24.53,8.49,5.66,10.38,52.83,33.96,...,0.86360,0.0,0.080,0.0,0.00255,0.019,0.000085,0.0000,1,1
17435,cinnamon raisin english muffins,NaN,United States,941.0,0.70,0.00,0.00,0.00,46.48,9.86,...,1.25222,0.0,0.000,0.0,0.00254,0.085,0.000000,0.0000,1,1
149893,pumpkin pecan spice muffin,NaN,United States,1377.0,16.47,1.18,9.41,4.12,41.18,17.65,...,0.32766,0.0,0.035,0.0,0.00085,0.071,0.001059,0.0000,1,1
281838,double chocolate muffin,Chocolate cake with plain and milk chocolate,Portugal,1766.0,22.40,4.10,12.10,6.00,52.20,26.90,...,1.27000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1
150993,double chocolate muffins,NaN,United States,1791.0,21.18,2.59,12.47,5.18,56.47,32.71,...,0.89154,0.0,0.000,0.0,0.00042,0.004,0.000000,0.0000,1,1


In [430]:
def pickK(clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    ratio = float(recall)/float(precision)
    print('Precision - %.3f' % precision)
    print('Recall - %.3f' % recall)
    print('Recall/Precision - %.3f' % ratio)
    print('train error - %.3f' % (1 - accuracy_score(y_train, clf.predict(X_train))))
    print('test error - %.3f' % (1 - accuracy_score(y_test, y_pred)))
    print('\n')
    
for i in range(1, 10):
    print(i)
    pickK(KNeighborsClassifier(i))

1
Precision - 0.506
Recall - 0.969
Recall/Precision - 1.914
train error - 0.000
test error - 0.062


2
Precision - 0.511
Recall - 0.854
Recall/Precision - 1.671
train error - 0.067
test error - 0.025


3
Precision - 0.506
Recall - 0.877
Recall/Precision - 1.731
train error - 0.083
test error - 0.047


4
Precision - 0.507
Recall - 0.877
Recall/Precision - 1.731
train error - 0.083
test error - 0.046


5
Precision - 0.505
Recall - 0.901
Recall/Precision - 1.783
train error - 0.083
test error - 0.065


6
Precision - 0.505
Recall - 0.901
Recall/Precision - 1.784
train error - 0.083
test error - 0.064


7
Precision - 0.505
Recall - 0.900
Recall/Precision - 1.782
train error - 0.100
test error - 0.067


8
Precision - 0.505
Recall - 0.902
Recall/Precision - 1.786
train error - 0.100
test error - 0.062


9
Precision - 0.503
Recall - 0.883
Recall/Precision - 1.754
train error - 0.150
test error - 0.101


